# AI Prompting Techniques

This notebook demonstrates various prompting techniques for interacting with AI models, specifically using OpenAI's GPT models.

In [3]:
from openai import OpenAI
import ollama
from typing import List
import json
import os
from dotenv import load_dotenv

load_dotenv()

# Define model constants
PROMPT_ONLY = False
USE_OLLAMA = True

OPENAI_CHAT_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"
OLLAMA_CHAT_MODEL = "llama3.2:latest"
OLLAMA_EMBEDDING_MODEL = "nomic-embed-text"

MAX_TOKENS = 2000
TEMPERATURE = 0.7

client: OpenAI


def initialize_openai_client() -> OpenAI:
    """Initialize and return an OpenAI client."""
    if not (api_key := os.getenv("OPENAI_API_KEY")):
        raise ValueError("OPENAI_API_KEY environment variable is not set")
    return OpenAI(api_key=api_key)


client = initialize_openai_client()


def get_llm_response(prompt: str, system_prompt: str = None) -> str:
    """Get a response from OpenAI's chat model."""
    if PROMPT_ONLY:
        return "Prompt only enabled for demonstration purposes."

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    else:
        messages.append(
            {
                "role": "system",
                "content": "You are a helpful assistant for the minor AI for Society. You respond in a concise manner.",
            }
        )
    messages.append({"role": "user", "content": prompt})

    if USE_OLLAMA:
        response = ollama.chat(
            model=OLLAMA_CHAT_MODEL,
            messages=messages,
        )
        return response["message"]["content"]
    else:
        response = client.chat.completions.create(
            model=OPENAI_CHAT_MODEL,
            messages=messages,
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
        )
        return response.choices[0].message.content


def get_embedding(text: str) -> List[float]:
    """Get an embedding from the embedding model."""
    if USE_OLLAMA:
        response = ollama.embed(model=OLLAMA_EMBEDDING_MODEL, input=[text])
        return response["embedding"]
    else:
        response = client.embeddings.create(input=[text], model=OPENAI_EMBEDDING_MODEL)
        return response.data[0].embedding


def print_prompt_and_response(technique_name: str, prompt: str, response: str):
    print(f"\n{'-'*20} {technique_name} {'-'*20}")
    print(f"Prompt:\n{prompt}")
    if not PROMPT_ONLY:
        print(f"\nResponse:\n{response}")
    print(f"{'-'*50}\n")

## Basic Prompting

Basic prompting is the simplest form of interaction with an Large Language Model. It works well for straightforward questions because it allows the model to use its general knowledge without constraints. However, it may lack specificity or structure in complex scenarios.

In [148]:
def basic_prompt_example():
    question = "What known percentage of responses from Large Language Models are hallucinations?"
    response = get_llm_response(question)
    print_prompt_and_response("Basic Prompt", question, response)


basic_prompt_example()


-------------------- Basic Prompt --------------------
Prompt:
What known percentage of responses from Large Language Models are hallucinations?

Response:
According to research, it's estimated that around 50-70% of the generated text by Large Language Models (LLMs) can be classified as "hallucinations" or entirely fabricated content. This is a significant concern, as many real-world applications rely on LLMs for accurate and trustworthy outputs.
--------------------------------------------------



## Zero-Shot Learning

Zero-shot learning tests the AI's ability to answer questions without specific examples. This technique works because it leverages the model's general knowledge and ability to understand context. It's useful for assessing the AI's baseline capabilities on various topics.

In [149]:
def zero_shot_example():
    question = "List the capital of each EU country neighbouring Germany"
    prompt = f"""
    Question: {question}
    
    Please provide a detailed answer without any additional context or examples.
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Zero-Shot Learning", prompt, response)


zero_shot_example()


-------------------- Zero-Shot Learning --------------------
Prompt:

    Question: List the capital of each EU country neighbouring Germany
    
    Please provide a detailed answer without any additional context or examples.
    

Response:
Here is the list of capitals for the European Union countries neighboring Germany:

1. Austria - Vienna
2. Belgium - Brussels
3. Czech Republic - Prague
4. Denmark - Copenhagen
5. France - Paris
6. Luxembourg - Luxembourg City
7. Netherlands - Amsterdam
8. Poland - Warsaw
--------------------------------------------------



## Few-Shot Learning

Few-shot learning provides the AI with examples before asking the main question.
This technique works well because it gives the model a pattern to follow, improving consistency and accuracy in responses. It's especially useful when you want the AI to mimic a specific response style or format.

In [150]:
def few_shot_example():
    question = "What is the capital of Europe?"
    prompt = f"""
    Here are a few examples:
    Q: What is the capital of Spain?
    A: The capital of Spain is Madrid.

    Q: What is the largest planet in our solar system?
    A: The largest planet in our solar system is Jupiter.

    Now, please answer the following question:
    Q: {question}
    A:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Few-Shot Learning", prompt, response)


few_shot_example()


-------------------- Few-Shot Learning --------------------
Prompt:

    Here are a few examples:
    Q: What is the capital of Spain?
    A: The capital of Spain is Madrid.

    Q: What is the largest planet in our solar system?
    A: The largest planet in our solar system is Jupiter.

    Now, please answer the following question:
    Q: What is the capital of Europe?
    A:
    

Response:
There is no single capital of Europe. Europe is a continent with many countries and no single city or country serves as its capital.
--------------------------------------------------



## Chain of Thought 

Chain of Thought prompting encourages the AI to break down its reasoning process. This technique is effective because it mimics human problem-solving, allowing for more transparent and logical responses. It's particularly useful for complex questions that require step-by-step analysis.

In [151]:
def chain_of_thought_example():
    question = (
        "What are the ethical considerations of using generative AI in education?"
    )
    prompt = f"""
    Question: {question}
    Let's approach this step-by-step:
    1) First, let's consider...
    2) Next, we should think about...
    3) Finally, we can conclude...

    Now, based on this reasoning, the answer is:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Chain of Thought", prompt, response)


chain_of_thought_example()


-------------------- Chain of Thought --------------------
Prompt:

    Question: What are the ethical considerations of using generative AI in education?
    Let's approach this step-by-step:
    1) First, let's consider...
    2) Next, we should think about...
    3) Finally, we can conclude...

    Now, based on this reasoning, the answer is:
    

Response:
Let's break down the ethical considerations of using generative AI in education step by step:

1. **Equity and Accessibility**: Generative AI tools may widen the gap between students with access to these tools and those without, potentially exacerbating existing inequalities in education.

2. **Authenticity and Original Work**: The use of generative AI raises questions about the authenticity and originality of student work, which can impact academic integrity and grading practices.

3. **Teacher Support and Training**: Educators may need training to effectively integrate generative AI tools into their teaching methods, ensuring

## Structured Output
Structured prompting provides a clear format and expectations for the AI's response.
This technique works well because it guides the model to organize information in a specific way, ensuring that all required elements are addressed. It's particularly useful for complex queries that require a systematic approach.

In [152]:
def structured_prompt_example():
    question = (
        "What are the ethical considerations of using generative AI in education?"
    )
    context = "Generative AI is increasingly being used in education to personalize learning experiences."
    format_requirements = {
        "application": "string",
        "benefits": "list",
        "challenges": "list",
    }
    evaluation_criteria = [
        "feasibility",
        "ethical considerations",
        "societal impact",
        "privacy",
        "bias",
        "legal considerations",
    ]

    prompt = f"""
    Context: {context}
    
    Question: {question}
    
    Required Format:
    {json.dumps(format_requirements, indent=2)}
    
    Evaluation Criteria: {', '.join(evaluation_criteria)}
    
    Provide your response following exactly the format above, considering the evaluation criteria.
    """

    response = get_llm_response(prompt)
    print_prompt_and_response("Structured Prompt", prompt, json.loads(response))


structured_prompt_example()


-------------------- Structured Prompt --------------------
Prompt:

    Context: Generative AI is increasingly being used in education to personalize learning experiences.
    
    Question: What are the ethical considerations of using generative AI in education?
    
    Required Format:
    {
  "application": "string",
  "benefits": "list",
  "challenges": "list"
}
    
    Evaluation Criteria: feasibility, ethical considerations, societal impact, privacy, bias, legal considerations
    
    Provide your response following exactly the format above, considering the evaluation criteria.
    

Response:
{'application': 'Generative AI in education is used to personalize learning experiences by creating customized content, assessing student performance, and providing real-time feedback.', 'benefits': ['Personalized learning experiences', 'Improved student engagement', 'Increased efficiency for teachers', 'Data-driven insights for informed instruction'], 'challenges': ['Bias in AI algori

## Role-Playing

Role-playing prompts the AI to adopt a specific persona or expertise.
This technique is effective because it narrows the model's focus to a particular perspective, leading to more specialized and contextually appropriate responses. It's useful for getting expert-like opinions on specific topics.

In [153]:
def role_playing_example():
    scenario = "A city is considering implementing AI-powered facial recognition in public spaces."
    role = "privacy advocate"
    system_prompt = f"You are an expert {role} working on AI applications in society. Provide responses from this perspective, considering ethical implications and societal impact."
    user_prompt = f"""
    Given the following scenario, provide your professional opinion and advice:

    Scenario: {scenario}

    Please provide your response in a professional manner, drawing from your expertise and addressing potential societal consequences.
    """
    response = get_llm_response(user_prompt, system_prompt=system_prompt)
    print_prompt_and_response(
        "Role-Playing", f"System: {system_prompt}\n\nUser: {user_prompt}", response
    )


role_playing_example()


-------------------- Role-Playing --------------------
Prompt:
System: You are an expert privacy advocate working on AI applications in society. Provide responses from this perspective, considering ethical implications and societal impact.

User: 
    Given the following scenario, provide your professional opinion and advice:

    Scenario: A city is considering implementing AI-powered facial recognition in public spaces.

    Please provide your response in a professional manner, drawing from your expertise and addressing potential societal consequences.
    

Response:
**Professional Opinion and Advice**

As a privacy advocate working on AI applications in society, I strongly advise against the implementation of AI-powered facial recognition in public spaces in this city. While the technology may offer some benefits, such as enhancing public safety or improving security, its risks to individual rights and freedoms far outweigh any potential advantages.

**Concerns and Risks**

1. **

## Persona-Based Approach

Persona-based prompting adapts the AI's response style and complexity to match different audience needs. This technique is effective because it ensures information is accessible and relevant to specific audiences. It's particularly useful in:

- Educational settings with diverse student backgrounds
- Business communications across different departments
- Technical documentation for various stakeholder groups
 
The key is specifying not just who the audience is, but also considering their context and likely frame of reference.

In [154]:
def persona_based_prompting_example():
    """Demonstrates how to get responses tailored to different audience levels."""
    topic = "How does machine learning work?"
    personas = [
        "a 10-year-old student with no background in technology",
        "a business executive of a mid-sized company in agriculture",
        "a computer science graduate with a focus on AI",
    ]

    for persona in personas:
        prompt = f"""
        Explain {topic} to {persona}.
        Keep the explanation appropriate for their background and use relevant analogies 
        they would understand based on their context.
        """
        response = get_llm_response(prompt)
        print_prompt_and_response(f"Persona-Based ({persona})", prompt, response)


persona_based_prompting_example()


-------------------- Persona-Based (a 10-year-old student with no background in technology) --------------------
Prompt:

        Explain How does machine learning work? to a 10-year-old student with no background in technology.
        Keep the explanation appropriate for their background and use relevant analogies 
        they would understand based on their context.
        

Response:
Imagine you have a huge library of books, and each book has a title, author, and some words written on it. You want to find a specific book with a certain title, but the problem is that you can't read all the titles and remember them.

Machine learning is like having a super smart librarian who helps you find the right book. Here's how it works:

1. **Training**: The librarian reads a lot of books (called "data") and writes down what they know about each book. For example, if a book has words like "cat" or "dog", the librarian will write them down.
2. **Practice**: The librarian practices recognizin

## Template-Based Approach

Template-based prompting uses predefined structures to ensure consistent and comprehensive responses. This technique is valuable because it:

- Maintains consistency across multiple queries
- Ensures all important aspects are covered
- Makes responses easier to process and compare
- Reduces the need for follow-up questions

It's particularly useful in business settings where standardized analysis is needed, or in research contexts where systematic evaluation is important.

In [155]:
def template_based_prompting_example():
    """Shows how to use templates for consistent responses."""
    template = """
    Topic: {topic}
    Context: {context}
    
    Please analyze this topic using the following structure:
    1. Definition:
       - Simple explanation
       - Key components
    
    2. Applications:
       - Current use cases
       - Potential future uses
    
    3. Implications:
       - Benefits
       - Challenges
       - Ethical considerations
    
    4. Recommendations:
       - Best practices
       - Implementation guidelines
    """

    example = {
        "topic": "Facial recognition in public spaces",
        "context": "Growing adoption of surveillance technologies in smart cities",
    }

    prompt = template.format(**example)
    response = get_llm_response(prompt)
    print_prompt_and_response("Template-Based", prompt, response)


template_based_prompting_example()


-------------------- Template-Based --------------------
Prompt:

    Topic: Facial recognition in public spaces
    Context: Growing adoption of surveillance technologies in smart cities
    
    Please analyze this topic using the following structure:
    1. Definition:
       - Simple explanation
       - Key components
    
    2. Applications:
       - Current use cases
       - Potential future uses
    
    3. Implications:
       - Benefits
       - Challenges
       - Ethical considerations
    
    4. Recommendations:
       - Best practices
       - Implementation guidelines
    

Response:
**Facial Recognition in Public Spaces: An Analysis**

**1. Definition:**

* Simple explanation: Facial recognition technology uses computer vision and machine learning algorithms to identify and recognize individuals based on their facial features.
* Key components: Computer vision, machine learning, database storage, and algorithmic analysis.

**2. Applications:**

* Current use cases:


## Comparative Analysis

Comparative analysis prompting helps in understanding relationships and trade-offs between different options. This technique is effective because it:

- Forces systematic comparison across multiple dimensions
- Highlights key differences and similarities
- Helps in decision-making processes
- Provides a comprehensive view of alternatives

It's especially useful when evaluating technologies, methodologies, or approaches where understanding relative strengths and weaknesses is crucial.

In [156]:
def comparative_analysis_prompting_example():
    """Demonstrates how to prompt for detailed comparisons."""
    prompt = """
    Compare and contrast the following AI technologies:
    1. Traditional Machine Learning
    2. Deep Learning
    3. Reinforcement Learning
    
    Use this structure:
    1. Key Characteristics
    2. Typical Applications
    3. Advantages
    4. Limitations
    5. Resource Requirements
    
    Present the comparison in a way that highlights the unique aspects of each approach
    and helps in understanding when to use which technology.
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Comparative Analysis", prompt, response)


comparative_analysis_prompting_example()


-------------------- Comparative Analysis --------------------
Prompt:

    Compare and contrast the following AI technologies:
    1. Traditional Machine Learning
    2. Deep Learning
    3. Reinforcement Learning
    
    Use this structure:
    1. Key Characteristics
    2. Typical Applications
    3. Advantages
    4. Limitations
    5. Resource Requirements
    
    Present the comparison in a way that highlights the unique aspects of each approach
    and helps in understanding when to use which technology.
    

Response:
Here's the comparison between Traditional Machine Learning, Deep Learning, and Reinforcement Learning:

**1. Key Characteristics**

*   **Traditional Machine Learning (ML):** Uses algorithms such as decision trees, linear regression, and clustering to make predictions or classify data.
    *   Linear models are suitable for small datasets with simple relationships between features.
    *   ML is often used in simple classification tasks like sentiment analysis

## Iterative Refinement

Iterative refinement prompting uses a series of connected prompts to progressively develop more detailed and nuanced responses. This technique is powerful because:

- It breaks complex topics into manageable chunks
- Each iteration builds on previous responses
- It allows for focused exploration of specific aspects
- It helps maintain context while diving deeper

This approach is particularly useful for complex topics where a single prompt might not capture all necessary details, or when different aspects need separate but connected analysis.

In [157]:
def iterative_refinement_prompting_example():
    """Shows how to iteratively improve responses through multiple prompts."""
    initial_question = (
        "What are the key considerations for implementing AI in healthcare?"
    )

    # Initial broad response
    initial_prompt = f"Provide a high-level overview: {initial_question}"
    initial_response = get_llm_response(initial_prompt)

    # First refinement - focus on technical aspects
    technical_prompt = f"""
    Based on this overview:
    {initial_response}
    
    Deep dive into the technical implementation challenges and requirements.
    Focus on infrastructure, data management, and system integration.
    """
    technical_response = get_llm_response(technical_prompt)

    # Second refinement - focus on ethical considerations
    ethical_prompt = f"""
    Based on this overview:
    {initial_response}
    
    And given the technical considerations:
    {technical_response}
    
    Analyze the ethical implications and necessary safeguards.
    Address privacy, bias, and patient rights.
    """
    ethical_response = get_llm_response(ethical_prompt)

    print_prompt_and_response(
        "Iterative Refinement - Initial", initial_prompt, initial_response
    )
    print_prompt_and_response(
        "Iterative Refinement - Technical", technical_prompt, technical_response
    )
    print_prompt_and_response(
        "Iterative Refinement - Ethical", ethical_prompt, ethical_response
    )


iterative_refinement_prompting_example()


-------------------- Iterative Refinement - Initial --------------------
Prompt:
Provide a high-level overview: What are the key considerations for implementing AI in healthcare?

Response:
Implementing AI in healthcare requires careful consideration of:

1. **Data quality and security**: Ensuring accurate, complete, and secure data to train and validate AI models.
2. **Regulatory compliance**: Adhering to laws and regulations, such as HIPAA, regarding patient data protection and medical device regulation.
3. **Clinical validation and testing**: Validating AI model performance, accuracy, and reliability in clinical settings.
4. **Ethics and bias**: Addressing potential biases and ensuring that AI systems prioritize fairness, transparency, and patient-centered care.
5. **Human-AI collaboration**: Designing AI systems to augment human expertise and decision-making, not replace it.
6. **Cybersecurity and continuity of care**: Protecting against cyber threats and ensuring seamless data ex

## Scenario-Based Approach

Scenario-based prompting provides rich context through realistic situations. This technique is effective because:

- It grounds abstract concepts in concrete situations
- It helps users understand practical applications
- It forces consideration of real-world constraints
- It makes responses more actionable and relevant

This approach is particularly valuable in training contexts, case studies, and when helping users understand how to apply concepts in practice. The key is providing enough detail to make the scenario realistic while keeping it focused on the learning objectives.

In [158]:
def scenario_based_prompting_example():
    """Demonstrates how to use realistic scenarios for context-rich responses."""
    scenario = """
    Scenario: MediCare Hospital is planning to implement an AI system for:
    1. Patient triage in the emergency department
    2. Predicting potential readmissions
    3. Scheduling staff based on predicted patient loads
    
    The hospital has:
    - 500 beds
    - 1,200 staff members
    - Electronic health records going back 5 years
    - A mix of urban and rural patients
    - Limited AI expertise on staff
    
    Required: Develop an implementation plan that addresses technical, 
    operational, and ethical considerations.
    """
    response = get_llm_response(scenario)
    print_prompt_and_response("Scenario-Based", scenario, response)


scenario_based_prompting_example()


-------------------- Scenario-Based --------------------
Prompt:

    Scenario: MediCare Hospital is planning to implement an AI system for:
    1. Patient triage in the emergency department
    2. Predicting potential readmissions
    3. Scheduling staff based on predicted patient loads
    
    The hospital has:
    - 500 beds
    - 1,200 staff members
    - Electronic health records going back 5 years
    - A mix of urban and rural patients
    - Limited AI expertise on staff
    
    Required: Develop an implementation plan that addresses technical, 
    operational, and ethical considerations.
    

Response:
Implementation Plan for MediCare Hospital's AI System:

**Phase 1: Planning (Weeks 1-4)**

1. **Assemble a multidisciplinary team**: Include hospital administrators, IT staff, medical professionals, and data analysts to ensure a comprehensive approach.
2. **Define project scope and objectives**: Determine the specific use cases for each AI system component (patient triage, r

## Task Decomposition

Task decomposition breaks down complex problems into smaller, manageable steps.
This technique works well because it helps the AI (and the user) approach complex issues systematically. It's particularly useful for planning or strategizing tasks that involve multiple components or stages.

In [159]:
def task_decomposition_example():
    complex_task = "Develop a plan to reduce carbon emissions in a major city using AI technologies."
    prompt = f"""
    Complex Task: {complex_task}

    Let's break this task down into smaller, manageable steps:

    1) [First step]
    2) [Second step]
    3) [Third step]
    ...

    Now, let's go through each step:

    1) [Explanation of first step]
    2) [Explanation of second step]
    3) [Explanation of third step]
    ...

    Conclusion: [Summary of the approach and final answer]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Task Decomposition", prompt, response)


task_decomposition_example()


-------------------- Task Decomposition --------------------
Prompt:

    Complex Task: Develop a plan to reduce carbon emissions in a major city using AI technologies.

    Let's break this task down into smaller, manageable steps:

    1) [First step]
    2) [Second step]
    3) [Third step]
    ...

    Now, let's go through each step:

    1) [Explanation of first step]
    2) [Explanation of second step]
    3) [Explanation of third step]
    ...

    Conclusion: [Summary of the approach and final answer]
    

Response:
I'd be happy to help you develop a plan to reduce carbon emissions in a major city using AI technologies. Here's the step-by-step plan:

**Step 1: Data Collection and Analysis**

* Collect and integrate data from various sources, including:
	+ Energy consumption patterns of households, businesses, and institutions
	+ Transportation modes (e.g., cars, buses, trains) and their carbon emissions
	+ Industrial processes and waste management practices
* Use AI algorith

## Self-Consistency

Self-consistency prompts the AI to generate multiple perspectives and then synthesize them. This technique is effective because it encourages a more comprehensive and balanced approach to a topic. It's particularly useful for complex issues where multiple viewpoints need to be considered.

In [160]:
def self_consistency_example():
    question = "What are the potential impacts of AI on job markets in the next decade?"
    prompt = f"""
    Question: {question}
    
    Please provide three different perspectives on this issue:
    
    Perspective 1:
    [Your response here]
    
    Perspective 2:
    [Your response here]
    
    Perspective 3:
    [Your response here]
    
    Now, synthesize these perspectives into a coherent answer:
    [Your synthesized response here]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Self-Consistency", prompt, response)


self_consistency_example()


-------------------- Self-Consistency --------------------
Prompt:

    Question: What are the potential impacts of AI on job markets in the next decade?
    
    Please provide three different perspectives on this issue:
    
    Perspective 1:
    [Your response here]
    
    Perspective 2:
    [Your response here]
    
    Perspective 3:
    [Your response here]
    
    Now, synthesize these perspectives into a coherent answer:
    [Your synthesized response here]
    

Response:
Here are the three different perspectives on the potential impacts of AI on job markets in the next decade:

**Perspective 1: Job Displacement and Economic Uncertainty**

The rise of AI could lead to significant job displacement, particularly in sectors where tasks are repetitive or can be easily automated. According to a report by McKinsey, up to 800 million jobs could be lost worldwide due to automation by 2030. This will lead to economic uncertainty, as workers struggle to adapt to new technologies an

## Constrained Generation 

Constrained generation sets specific limits or requirements for the AI's response. This technique works well because it challenges the AI to be creative within defined boundaries, often resulting in more focused and precise outputs. It's useful when you need responses in a very specific format or style.

In [161]:
def constrained_generation_example():
    scenario = "A city is implementing an AI-powered traffic light system."
    prompt = f"""
    Scenario: {scenario}
    
    Please provide a response that meets the following constraints:
    1. Use exactly 50 words.
    2. Include at least one potential benefit and one potential risk.
    3. End with a question for further consideration.
    
    Your response:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Constrained Generation", prompt, response)


constrained_generation_example()


-------------------- Constrained Generation --------------------
Prompt:

    Scenario: A city is implementing an AI-powered traffic light system.
    
    Please provide a response that meets the following constraints:
    1. Use exactly 50 words.
    2. Include at least one potential benefit and one potential risk.
    3. End with a question for further consideration.
    
    Your response:
    

Response:
The AI-powered traffic light system aims to optimize traffic flow, reducing congestion and travel time. Potential benefits include increased safety and reduced emissions. However, a risk is the dependence on complex algorithms, which may lead to system failures or biases if not designed with robust data validation and testing protocols.
--------------------------------------------------



## Socratic Method - Simulation

In [162]:
def socratic_method_example():
    topic = "The impact of social media on mental health"
    initial_question = "What are the primary ways social media affects mental health?"

    questions = [initial_question]
    answers = []

    for i in range(3):  # Let's do 3 rounds of questions
        current_question = questions[-1]
        prompt = f"""
        Topic: {topic}
        Previous questions: {questions[:-1]}
        Previous answers: {answers}
        
        Current question: {current_question}
        
        Please provide a short and concise answer to the current question, and then ask a follow-up question 
        that digs deeper into the topic, challenging assumptions or exploring implications of the answer.
        
        Format your response as:
        Answer: [Your answer here]
        Thoughts: [Why you think this way]
        Possible counterarguments: [Possible counterarguments to your answer]
        Follow-up question: [Your follow-up question here]
        """

        response = get_llm_response(prompt)
        print_prompt_and_response(f"Socratic Method - Round {i+1}", prompt, response)

        # Parse the response to get the answer and next question
        if PROMPT_ONLY:
            answer = f"Dummy answer {i+1}"
            next_question = f"Dummy next question {i+1}"
        else:
            answer, next_question = response.split("Follow-up question:")
            answer = answer.strip().replace("Answer: ", "")
            next_question = next_question.strip()

        answers.append(answer)
        questions.append(next_question)


socratic_method_example()


-------------------- Socratic Method - Round 1 --------------------
Prompt:

        Topic: The impact of social media on mental health
        Previous questions: []
        Previous answers: []
        
        Current question: What are the primary ways social media affects mental health?
        
        Please provide a short and concise answer to the current question, and then ask a follow-up question 
        that digs deeper into the topic, challenging assumptions or exploring implications of the answer.
        
        Format your response as:
        Answer: [Your answer here]
        Thoughts: [Why you think this way]
        Possible counterarguments: [Possible counterarguments to your answer]
        Follow-up question: [Your follow-up question here]
        

Response:
Answer: Social media affects mental health primarily through the comparison trap, decreased face-to-face interaction, and increased exposure to curated and unrealistic content.

Thoughts: The primary ways

## Reflective Prompting - Simulation

Reflective prompting encourages the AI to critically examine its own responses. This technique is effective because it promotes a more thoughtful and comprehensive analysis, addressing potential shortcomings in the initial response. It's particularly useful for complex topics where considering multiple perspectives and acknowledging limitations is important.

In [163]:
def reflective_prompting_example():
    initial_question = "How can AI be used to address climate change?"
    prompt = f"""
    Initial question: {initial_question}

    Please provide an initial response to this question. Then, reflect on your answer by considering:
    1. What assumptions did you make in your response?
    2. Are there any potential biases in your answer?
    3. What additional information would be helpful to provide a more comprehensive response?
    4. How might different stakeholders view this solution differently?

    After reflection, provide an updated, more nuanced response.

    Format your answer as follows:
    Initial response: [Your initial response here]

    Reflection:
    1. Assumptions: [List assumptions]
    2. Potential biases: [Discuss potential biases]
    3. Additional information needed: [List additional information]
    4. Stakeholder perspectives: [Discuss different viewpoints]

    Updated response: [Your updated, more nuanced response here]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Reflective Prompting", prompt, response)


reflective_prompting_example()


-------------------- Reflective Prompting --------------------
Prompt:

    Initial question: How can AI be used to address climate change?

    Please provide an initial response to this question. Then, reflect on your answer by considering:
    1. What assumptions did you make in your response?
    2. Are there any potential biases in your answer?
    3. What additional information would be helpful to provide a more comprehensive response?
    4. How might different stakeholders view this solution differently?

    After reflection, provide an updated, more nuanced response.

    Format your answer as follows:
    Initial response: [Your initial response here]

    Reflection:
    1. Assumptions: [List assumptions]
    2. Potential biases: [Discuss potential biases]
    3. Additional information needed: [List additional information]
    4. Stakeholder perspectives: [Discuss different viewpoints]

    Updated response: [Your updated, more nuanced response here]
    

Response:
Initia

## Guided Feedback - Simulation

Guided feedback prompting involves providing specific feedback or guidance to the AI after its initial response. This technique allows the AI to refine and improve its answer based on targeted suggestions. It's particularly useful for obtaining more comprehensive, nuanced responses and for directing the AI's focus to specific aspects of a complex topic.

In [164]:
def guided_feedback_prompting_example():
    initial_question = (
        "What are the potential impacts of AI on job markets in the next decade?"
    )
    prompt = f"""
    Initial question: {initial_question}

    Please provide an initial response to this question.
    """
    initial_response = get_llm_response(prompt)

    feedback_prompt = f"""
    Your initial response was:
    {initial_response}

    This response is a good start, but let's improve it with the following guidance:
    1. Consider more diverse scenarios, including edge cases.
    2. Discuss both short-term and long-term risks.
    3. Include potential mitigation strategies for each risk.
    4. Address the ethical implications of these risks.

    Please respond by responding to the feedback then providing an updated response incorporating this feedback.
    """
    updated_response = get_llm_response(feedback_prompt)

    print_prompt_and_response(
        "Guided Feedback Prompting",
        f"{feedback_prompt[:300]}...\n\nUpdated Response after feedback:",
        updated_response,
    )


guided_feedback_prompting_example()


-------------------- Guided Feedback Prompting --------------------
Prompt:

    Your initial response was:
    The potential impact of AI on job markets in the next decade is multifaceted and varied. Some possible effects include:

1. **Job displacement**: AI may displace certain jobs, especially those that involve repetitive or routine tasks, such as data entry, customer ...

Updated Response after feedback:

Response:
**Revised Response: The Potential Impact of AI on Job Markets in the Next Decade**

The potential impact of AI on job markets in the next decade is multifaceted and varied. Some possible effects include:

1. **Job displacement**: AI may displace certain jobs, especially those that involve repetitive or routine tasks, such as:
	* Data entry clerks
	* Customer service representatives with limited task complexity
	* Manufacturing line workers
	* Short-term risks: Job loss within the next 5-10 years
	* Long-term risks: Job displacement may accelerate if not addressed thro

## Verification Prompting

Modern self-verification prompting has evolved beyond simple rechecking. It now incorporates multiple verification strategies:

- Breaking down complex problems into verifiable components
- Using different methods to cross-validate results
- Applying logical constraints and edge cases
- Leveraging visual representations for verification
- Systematic error checking

This technique is particularly effective because it:

- Reduces errors in complex calculations
- Helps identify hidden assumptions
- Provides multiple perspectives on the same problem
- Forces systematic thinking about edge cases
- Creates more reliable and trustworthy outputs

It's especially useful in:

- Mathematical calculations
- Logical reasoning problems
- Data analysis tasks
- Decision-making scenarios
- Any situation where accuracy is crucial

The key is to structure the verification process so it's not just repeating the same calculation, but approaching the problem from different angles to ensure robustness of the solution.

### Self Verification

In [167]:
def self_verification_prompting_example():
    """Demonstrates advanced self-verification prompting technique."""
    problem = """
    In a class of 30 students:
    - 2/5 of the students play soccer
    - half of the students play basketball
    - a quarter of the students play both sports
    - 1/3 of the students play guitar and 2 of them play both sports and guitar
    
    How many students don't play either sport and play guitar?
    """

    verification_prompt = f"""
    Problem: {problem}
    
    Please solve this problem following these steps:
    1. First, provide your initial calculation
    2. Then verify your work by:
       - Breaking down each component
       - Checking your assumptions
       - Identifying potential errors
       - Recalculating using a different method
       - Comparing both results
    3. If you find any discrepancies, explain and correct them
    4. Critique the challenge of the problem if you think it's stated incorrectly
    5. Provide your final, verified answer
    
    Show all your work and reasoning. Readable text format, no code.
    """
    response = get_llm_response(verification_prompt)
    print_prompt_and_response("Self-Verification", verification_prompt, response)


self_verification_prompting_example()


-------------------- Self-Verification --------------------
Prompt:

    Problem: 
    In a class of 30 students:
    - 2/5 of the students play soccer
    - half of the students play basketball
    - a quarter of the students play both sports
    - 1/3 of the students play guitar and 2 of them play both sports and guitar
    
    How many students don't play either sport and play guitar?
    
    
    Please solve this problem following these steps:
    1. First, provide your initial calculation
    2. Then verify your work by:
       - Breaking down each component
       - Checking your assumptions
       - Identifying potential errors
       - Recalculating using a different method
       - Comparing both results
    3. If you find any discrepancies, explain and correct them
    4. Critique the challenge of the problem if you think it's stated incorrectly
    5. Provide your final, verified answer
    
    Show all your work and reasoning. Readable text format, no code.
    

Respo

### Logical Verification

In [166]:
def logical_verification_prompting_example():
    """Shows how to use verification through logical constraints."""
    query = """
    In a class of 30 students:
    - 2/5 of the students play soccer
    - half of the students play basketball
    - a quarter of the students play both sports
    - 1/3 of the students play guitar and 2 of them play both sports and guitar
    
    How many students don't play either sport and play guitar?
    """

    prompt = f"""
    Problem: {query}
    
    Please solve this problem using the following verification framework:
    
    1. Initial Solution:
       - Show your calculation
       - Explain your reasoning
    
    2. Verify Logical Constraints:
       - Total students must equal 30
       - Students in both sports ≤ minimum of individual sports
       - Total students in either sport ≤ sum of individual sports
       - Final answer cannot be negative
    
    3. Visual Verification:
       - Draw a Venn diagram (describe it in text)
       - Verify numbers match across all representations
    
    4. Edge Case Check:
       - What if all numbers were equal?
       - What if there was no overlap?
       - Do these cases reveal any issues in your logic?
       
    5. Critique the challenge of the problem if you think it's stated incorrectly
    
    6. Final Verified Answer:
       - State your conclusion
       - Confirm all constraints are satisfied
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Logical Verification", prompt, response)


logical_verification_prompting_example()


-------------------- Logical Verification --------------------
Prompt:

    Problem: 
    In a class of 30 students:
    - 2/5 of the students play soccer
    - half of the students play basketball
    - a quarter of the students play both sports
    - 1/3 of the students play guitar and 2 of them play both sports and guitar
    
    How many students don't play either sport and play guitar?
    
    
    Please solve this problem using the following verification framework:
    
    1. Initial Solution:
       - Show your calculation
       - Explain your reasoning
    
    2. Verify Logical Constraints:
       - Total students must equal 30
       - Students in both sports ≤ minimum of individual sports
       - Total students in either sport ≤ sum of individual sports
       - Final answer cannot be negative
    
    3. Visual Verification:
       - Draw a Venn diagram (describe it in text)
       - Verify numbers match across all representations
    
    4. Edge Case Check:
       

# Problem Solving Clarification

This prompt is designed to guide an AI in methodically checking and clarifying problem statements before attempting solutions. It is effective because:

1. Sequential Clarification:
   - Handles one issue at a time to avoid overwhelming the user
   - Prioritizes the most critical ambiguities first
   - Maintains a clear dialogue flow

2. Problem-Solving Structure:
   - Starts with a thorough analysis of the problem statement
   - Identifies potential issues before attempting a solution
   - Ensures all necessary information is available

3. Interactive Approach:
   - Creates a natural back-and-forth dialogue
   - Allows for iterative clarification
   - Builds understanding progressively

4. Clear Communication:
   - Explains why each clarification is needed
   - Acknowledges received information
   - Maintains context throughout the conversation

This prompt is particularly useful for:

- Mathematical word problems
- Logic puzzles
- Complex scenario questions
- Multi-step problems
- Any problem where assumptions need verification

The key is preventing the common issue of AIs making assumptions or proceeding with incomplete information, which can lead to incorrect solutions.

In [31]:
def join_list(items: List[str]) -> str:
    """Join list items with newlines, adding numbers."""
    return (
        "\n".join(f"\n - {i+1}. {item}" for i, item in enumerate(items))
        if items
        else ""
    )


def generate_problem_solving_clarifications(
    questions: List[str] = [], answers: List[str] = []
) -> str:
    return f"""
    Problem to solve: 
      - 2/5 of the students play soccer
      - half of the students play basketball
      - a quarter of the students play both sports
      - 1/3 of the students play guitar and 2 of them play both sports and guitar
    How many students don't play either sport and play guitar?
    
    Your questions: 
    {join_list(questions)}
    
    My answers to your questions: 
    {join_list(answers)}

    Before solving this problem:
    1. First check if there are any ambiguities, missing information, or inconsistencies in the problem.
    
    2. If you find any issues:
       - Ask ONE clear, specific question to resolve the most critical ambiguity
       - Explain why this clarification is needed
       - Wait for an answer before proceeding further
       
    3. If the problem is completely clear:
       - State that no clarification is needed
       - Proceed with solving the problem step by step

    4. After receiving any clarification:
       - Acknowledge the clarification
       - Check if any other critical ambiguities remain
       - If clear, proceed with the solution
       - If not, ask the next most important question

    Remember: Ask only ONE question at a time, focusing on the most critical issue first.
    """


# Example usage
responses = []

prompt = generate_problem_solving_clarifications(None, None)
response = get_llm_response(prompt)
responses.append(response)

print_prompt_and_response("Problem Solving", prompt, response)

prompt = generate_problem_solving_clarifications(responses, ["Maybe"])
response = get_llm_response(prompt)
responses.append(response)

print_prompt_and_response("Problem Solving", prompt, response)

prompt = generate_problem_solving_clarifications(
    responses, ["Maybe", "I really don't know, sorry. Ok, yes."]
)
response = get_llm_response(prompt)
responses.append(response)

print_prompt_and_response("Problem Solving", prompt, response)

prompt = generate_problem_solving_clarifications(
    responses, ["Maybe", "I really don't know, sorry. Ok, yes.", "YES!"]
)
response = get_llm_response(prompt)
responses.append(response)

print_prompt_and_response("Problem Solving", prompt, response)


-------------------- Problem Solving --------------------
Prompt:

    Problem to solve: 
      - 2/5 of the students play soccer
      - half of the students play basketball
      - a quarter of the students play both sports
      - 1/3 of the students play guitar and 2 of them play both sports and guitar
    How many students don't play either sport and play guitar?
    
    Your questions: 
    
    
    My answers to your questions: 
    

    Before solving this problem:
    1. First check if there are any ambiguities, missing information, or inconsistencies in the problem.
    
    2. If you find any issues:
       - Ask ONE clear, specific question to resolve the most critical ambiguity
       - Explain why this clarification is needed
       - Wait for an answer before proceeding further
       
    3. If the problem is completely clear:
       - State that no clarification is needed
       - Proceed with solving the problem step by step

    4. After receiving any clarificati